In [1]:
from torch import optim
from lib.datasets import get_stock_price,train_test_split
from lib.aug import apply_augmentations,parse_augmentations,sig_normal
import torch
from torch import nn
import torch.nn.functional as F
from lib.utils import set_seed
import signatory
from models.betavae import BetaVAE, BetaVAE_train
from models.infovae import InfoVAE, InfoVAE_train
from models.cvae import CVAE, CAVE_train
from lib.metrics import mmd_loss, kl_divergence

In [2]:
set_seed(0)

In [3]:
data_config = {
    "ticker" : "^GSPC",
    "interval" : "1d",
    "column" : 1,  
    "window_size" : 20,
    "dir" : "datasets",
    "subdir" : "stock"
}
sig_config = {
    "augmentations": [{"name": "LeadLag"}],
    "device" : "cuda",
    "depth" : 3,
}

In [4]:
if sig_config["augmentations"] is not None:
    sig_config["augmentations"] = parse_augmentations(sig_config.get('augmentations'))

In [5]:
tensor_data = get_stock_price(data_config)
x_real_train, x_real_test = train_test_split(tensor_data, train_test_ratio=0.8, device=sig_config["device"])
print("x_real_train shape {}, x_real_test shape {}".format(x_real_train.shape,x_real_test.shape))

if sig_config["augmentations"] is not None:
    # Print the tensor shape after each augmentation
    x_aug_train, x_aug_test = apply_augmentations(x_real_train,sig_config["augmentations"]), apply_augmentations(x_real_test,sig_config["augmentations"])
print("After augmentation shape:",x_aug_train.shape)

# To signature
x_sig_train, x_sig_test = signatory.signature(x_aug_train,sig_config["depth"]),signatory.signature(x_aug_test,sig_config["depth"])
print("x_sig_train shape {}, x_sig_test shape {}".format(x_sig_train.shape,x_sig_test.shape))
input_dim = x_sig_train.shape[1]
print("input_dim: {}".format(input_dim))

# Normalize
x_sig_train, x_sig_test = sig_normal(x_sig_train,True), sig_normal(x_sig_test,True)

# To device
x_sig_train, x_sig_test = x_sig_train.to(sig_config["device"]), x_sig_test.to(sig_config["device"])

Rolled data for training, shape torch.Size([1232, 20, 1])
x_real_train shape torch.Size([985, 20, 1]), x_real_test shape torch.Size([247, 20, 1])
torch.Size([985, 39, 2])
torch.Size([247, 39, 2])
After augmentation shape: torch.Size([985, 39, 2])
x_sig_train shape torch.Size([985, 14]), x_sig_test shape torch.Size([247, 14])
input_dim: 14


In [6]:
model_config = {
    "hidden_dims" : [input_dim,8,2],
    "device" : "cuda:0",
}

In [7]:
model_betavae = BetaVAE(x_sig_train,6000,128,beta=4,device=model_config["device"],hidden_dims=model_config["hidden_dims"])
optimizer = torch.optim.Adam(model_betavae.parameters(), lr=1e-4)
BetaVAE_train(model=model_betavae,optimizer=optimizer)

Input tensor shape: torch.Size([985, 14])
Hidden dims: [14, 8, 2]
Beta: 4
Epoch 0 loss 16.324797
Epoch 100 loss 6.576211
Epoch 200 loss 7.512055
Epoch 300 loss 18.123297
Epoch 400 loss 6.085393
min_loss: 4.209315


In [8]:
model_config = {
    "hidden_dims" : [input_dim,7,3],
    "device" : "cuda:0",
}

In [9]:
model_betavae = BetaVAE(x_sig_train,6000,128,beta=4,device=model_config["device"],hidden_dims=model_config["hidden_dims"])
optimizer = torch.optim.Adam(model_betavae.parameters(), lr=1e-4)
BetaVAE_train(model=model_betavae,optimizer=optimizer)

Input tensor shape: torch.Size([985, 14])
Hidden dims: [14, 7, 3]
Beta: 4
Epoch 0 loss 5.157246
Epoch 100 loss 39.749989
Epoch 200 loss 29.868149
Epoch 300 loss 8.678584
Epoch 400 loss 15.197567
min_loss: 3.565665


In [10]:
model_config = {
    "hidden_dims" : [input_dim,10,5],
    "device" : "cuda:0",
}

Train

In [11]:
model_betavae = BetaVAE(x_sig_train,6000,128,beta=4,device=model_config["device"],hidden_dims=model_config["hidden_dims"])
optimizer = torch.optim.Adam(model_betavae.parameters(), lr=1e-4)
BetaVAE_train(model=model_betavae,optimizer=optimizer)

Input tensor shape: torch.Size([985, 14])
Hidden dims: [14, 10, 5]
Beta: 4
Epoch 0 loss 18.806564
Epoch 100 loss 30.169491
Epoch 200 loss 7.456823
Epoch 300 loss 15.236901
Epoch 400 loss 31.009821
min_loss: 4.257915


Test

In [ ]:
# Use x_sig_test to test the model
_, _, z = model_betavae.encode(x_sig_test)
reconstructed_sig = model_betavae.decode(z) 

KLD = nn.KLDivLoss(reduction="batchmean")

# Mean of L2 norm over the batch dimension
beta_L2 = F.mse_loss(x_sig_test,reconstructed_sig)
# Mean of JS divergence over the batch dimension
beta_js = (KLD(x_sig_test,reconstructed_sig)+KLD(reconstructed_sig,x_sig_test))/2
# signature MMD
beta_mmd = mmd_loss(x_sig_test,reconstructed_sig)

print("L2: {:4f}".format(beta_L2.item()))
print("KL: {:4f}".format(beta_js.item()))
print("mmd: {:4f}".format(beta_mmd.item()))

L2: 1.044617
KL: -1.686580
mmd: 0.368814
